In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd '/content/drive/MyDrive/'

    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/MyDrive
Thu May 12 20:01:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

df = pd.read_csv('/content/drive/MyDrive/Data Mining/dataset.txt')

df_target = df['humor']
df_data = df.copy()
df_data.drop(columns='humor')

df_target.head()

0    False
1    False
2     True
3    False
4    False
Name: humor, dtype: bool

In [3]:
from sklearn import preprocessing

#encode target to numeric
label_encoder = preprocessing.LabelEncoder()
df_target = label_encoder.fit_transform(df_target)
df_target

array([0, 0, 1, ..., 1, 0, 1])

In [4]:
words=20000
max_length=1000

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

#Apply CountVectorizer on text column

#vectorizer = CountVectorizer(binary=True)

#If this vectorizer is used, a word which occurs multiple times is counted and not just displayed as 1 = word occurs
cVectorizer = CountVectorizer(binary=False, max_features=1000) #  restrict feature size --> may need to play around

#At the moment I am not able to apply the vectorizer on the whole dataset. To many records & less available RAM
matrix = cVectorizer.fit_transform(df_data['text']).toarray()

In [6]:
from sklearn.model_selection import train_test_split

df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    matrix, df_target, test_size=0.2, random_state=42)

In [7]:
word_size=words
word_size

embed_size=128

In [8]:
model=tf.keras.Sequential()

# Embedding Layer
model.add(tf.keras.layers.Embedding(word_size, embed_size, input_shape=(df_data_train.shape[1],)))

# LSTM Layer
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

# Output Layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [9]:
type(df_data_test)

numpy.ndarray

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 128)         2560000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
"""#### Training the model"""

model.fit(df_data_train, df_target_train, epochs=5, batch_size=512)

Epoch 1/5
313/313 [==============================] - 57s 162ms/step - loss: 0.6631 - accuracy: 0.6085
Epoch 2/5
313/313 [==============================] - 51s 162ms/step - loss: 0.5964 - accuracy: 0.6912
Epoch 3/5
313/313 [==============================] - 51s 162ms/step - loss: 0.5675 - accuracy: 0.7227
Epoch 4/5
313/313 [==============================] - 51s 162ms/step - loss: 0.5591 - accuracy: 0.7255
Epoch 5/5
313/313 [==============================] - 51s 163ms/step - loss: 0.5492 - accuracy: 0.7295


In [14]:
test_loss, test_acurracy = model.evaluate(df_data_test, df_target_test)

print("Test accuracy: {}".format(test_acurracy))

1250/1250 [==============================] - 24s 18ms/step - loss: 0.5376 - accuracy: 0.7338
Test accuracy: 0.7337750196456909
